<a href="https://colab.research.google.com/github/ghluque/Analisis_BTC/blob/master/BTC_TX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero importamos las librerias que vamos a necesitar 

Requests se utiliza para leer informacion desde una url
json es un tipo de archivo, pandas solo lee json con forma de data frame. 

In [45]:
import pandas as pd
import numpy as np

import requests 
import json 


Nombramos el url de block

In [46]:
block_url="https://blockchain.info/rawblock/0000000000000000000cfe48631b9dc8083201f7bda4deb5bc081e344d1a5913"

block es un diccionario de diccionarios y arrays

In [47]:
block= json.loads(requests.get(block_url).text) 

Por ej: la transaccion 2, que propiedades tiene? 

In [48]:
block['tx'][2] 

{'balance': 0,
 'block_height': 619002,
 'block_index': 0,
 'double_spend': False,
 'fee': 100000,
 'hash': '12565431a8f14cc4e65767b76914969474652fa0b69b02e8fcf177f977e9e5d7',
 'inputs': [{'index': 0,
   'prev_out': {'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
    'n': 2,
    'script': '76a91443849383122ebb8a28268a89700c9f723663b5b888ac',
    'spending_outpoints': [{'n': 0, 'tx_index': 0}],
    'spent': True,
    'tx_index': 0,
    'type': 0,
    'value': 253537196},
   'script': '483045022100d981306e2cb0ef59167c408d3dbd81e81af36e9e105ccbf0d110b3d8af39c6f50220246b4e8e7e36be43148778ea6fb8658728535fcdc31503fb821d083f71654d9d0141047146f0e0fcb3139947cf0beb870fe251930ca10d4545793d31033e801b5219abf56c11a3cf3406ca590e4c14b0dab749d20862b3adc4709153c280c2a78be10c',
   'sequence': 4294967295,
   'witness': ''}],
 'lock_time': 0,
 'out': [{'addr': '325kjwqJKbcCnJ2LhjqmmeXaPhYiTZ1EFN',
   'n': 0,
   'script': 'a914044ce4aa9472b406557ae47b52b98d098dc29d7a87',
   'spending_outpoints': [{'n': 1, 't

Maximo de inputs por tx

Maximo de outputs por tx

Agrega 0 a la lista hasta alcanzar el desiredLen

Devuelve una tx de la blockchain como una lista con hash, value para 5 inputs y 5 value outputs.

In [49]:
IN_CNT_MAX=5 #U: maximo de inputs por tx
OUT_CNT_MAX=5 #U: maximo de outputs por tx
Tx_keys= ['hash','time']
Tx_normalized_cols= Tx_keys + ['i'+str(n)+'v' for n in range(0,IN_CNT_MAX)] + ['o'+str(n)+'v' for n in range(0,OUT_CNT_MAX)]
def pad_list(l,desiredLen):
  
  return l + [0] * (desiredLen-len(l)) if len(l)<desiredLen else l[0:desiredLen]

def tx_in_normalized(tx):
  return [(x['prev_out']['value'] if 'prev_out' in x else 0) for x in tx['inputs']]

def tx_out_normalized(tx):
  return [x['value'] for x in tx['out']]

def tx_normalized(tx):
  
  return [tx[k] for k in Tx_keys] + pad_list(tx_in_normalized(tx),IN_CNT_MAX) + pad_list(tx_out_normalized(tx),OUT_CNT_MAX)

Normalizamos nuestro bloque para que tenga forma de tabla y creamos un dataframe

Convertimos el time del bloque en un date time.

In [50]:
tx= pd.DataFrame( [ tx_normalized(tx) for tx in block['tx'] ], columns= Tx_normalized_cols)

tx.time= pd.to_datetime(tx.time, unit='s') 

Transaccion: x ej si en una anterior compre 3 btc y le pago 1 a Mauricio. Le mando 1 btc a Mauricio y 2 a mi billetera (que seria el vuelto)

In [51]:
print("Direccion anterior {}\nEl vuelto va a {}\n",[ block['tx'][2]['inputs'][0]['prev_out']['addr'], block['tx'][2]['out'][2]['addr'] ])

Direccion anterior {}
El vuelto va a {}
 ['17A16QmavnUfCW11DAApiJxp7ARnxN5pGX', '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX']


Los numeros del tx estan en "satoshis" que son la millonesima parte de 1 btc. Entonces procedemos a dividir por /1000000 para obtener numeros mas legibles

Sumamos para conocer el total en bruto de cada transaccion. Y de paso le agregamos la columna TOTAL al data frame. 

Y por ultimo, mostramos el encabezado.

In [55]:
#imputs
tx['i0v'] /= 1000000
tx['i1v'] /= 1000000
tx['i2v'] /= 1000000
tx['i3v'] /= 1000000
tx['i4v'] /= 1000000


#outputs
tx['o0v'] /= 1000000
tx['o1v'] /= 1000000
tx['o2v'] /= 1000000
tx['o3v'] /= 1000000
tx['o4v'] /= 1000000



tx['TOTAL']=tx.iloc[:,2:11].sum(axis=1)


tx.head(20)

,hash,time,i0v,i1v,i2v,i3v,i4v,o0v,o1v,o2v,o3v,o4v,TOTAL
0,bfcc1c233827fbf147b9744b58844465abf61232c02fb2...,2020-02-26 00:10:06,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.276409e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.276409e-15
1,8325d3decbb805e09f042fc946bc61076be3578af331b1...,2020-02-26 00:10:06,5.000000e-14,0.000000e+00,0.000000e+00,0.0,0.0,5.100000e-15,4.489992e-14,0.000000e+00,0.000000e+00,0.000000e+00,9.999992e-14
2,12565431a8f14cc4e65767b76914969474652fa0b69b02...,2020-02-26 00:10:06,2.535372e-16,0.000000e+00,0.000000e+00,0.0,0.0,1.180000e-18,7.125764e-18,2.451314e-16,0.000000e+00,0.000000e+00,5.069744e-16
3,0f24da007c306726713906933e0e171800290cbf5e3047...,2020-02-26 00:10:06,2.451314e-16,0.000000e+00,0.000000e+00,0.0,0.0,7.875000e-18,9.994323e-18,2.271621e-16,0.000000e+00,0.000000e+00,4.901629e-16
4,9491757c01ab9c01eb03d6db97e4feba4c4675bf6dd803...,2020-02-26 00:10:06,7.778833e-16,0.000000e+00,0.000000e+00,0.0,0.0,1.070052e-17,7.671327e-16,0.000000e+00,0.000000e+00,0.000000e+00,1.555716e-15
5,24bfcbbe7cf1758c83172158f56c33055f129cf327f29f...,2020-02-26 00:10:06,2.931460e-18,4.064356e-18,0.000000e+00,0.0,0.0,3.685800e-19,6.515696e-18,0.000000e+00,0.000000e+00,0.000000e+00,1.388009e-17
6,864d6e85a9ad07f6a678971c2fb625935c01df4393abb7...,2020-02-26 00:10:06,8.783807e-17,0.000000e+00,0.000000e+00,0.0,0.0,8.757472e-17,2.133480e-19,0.000000e+00,0.000000e+00,0.000000e+00,1.756261e-16
7,7a82cf2b88313314a7a498534b839e608fa430baa018c7...,2020-02-26 00:10:06,9.934895e-17,0.000000e+00,0.000000e+00,0.0,0.0,9.842795e-17,8.710000e-19,0.000000e+00,0.000000e+00,0.000000e+00,1.986479e-16
8,d7773033a2af0d7aacb40e3cdd07beb2b7c1a09a088236...,2020-02-26 00:10:06,7.776820e-19,1.000000e-16,1.008317e-18,0.0,0.0,1.000000e-16,1.716399e-18,0.000000e+00,0.000000e+00,0.000000e+00,2.035024e-16
9,d71168c5cc0be93fdc1307cd79a1c77783ccdac9a15126...,2020-02-26 00:10:06,3.829100e-16,0.000000e+00,0.000000e+00,0.0,0.0,1.884200e-16,1.944568e-16,0.000000e+00,0.000000e+00,0.000000e+00,7.657869e-16


In [61]:
tx.groupby(['addr'])

KeyError: ignored

resultado = df.groupby("Nombre").apply(
    lambda x: pd.Series(encontrar_fechas(rango, x.Fecha))
)Sacamos la media para considerar que los que posean + de la media sean considerados GRANDES y los que tengan - sean considerados CHICOS


O sea que de 2830 transacciones, solo 397 superan la media